In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import sys

In [5]:
sys.path.insert(0, '../gahyparopt/')
sys.path.insert(0, '../evolution-und-ki')

In [6]:
from gahyperopt import *

In [7]:
import ipywidgets as widgets
import os, shutil, shlex
import json, io
from subprocess import Popen, PIPE, STDOUT
from contextlib import redirect_stdout, redirect_stderr
from datetime import datetime
import tensorflow as tf
import pandas
from matplotlib import pyplot
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from keras import backend as K
import glob
from collections import OrderedDict

In [17]:
from GAUtilities import *

In [9]:
from parameters import *

In [10]:
def load_population():
    json_fnames = glob.glob("*.json")
    population = []
    names = []
    for j in json_fnames:
        name = j.split(".")[0]
        names.append(name)
        individuum = read_chromosome(j.split('.')[0])
        population.append(individuum)
        
    population = OrderedDict(zip(names, population))
    population=OrderedDict(sorted(population.items(),key=lambda x: x[1].accuracy, reverse=True))
    return population

In [11]:
POPULATION_SIZE=6

In [12]:
ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=POPULATION_SIZE,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        )

In [13]:
class model(object):
    generation_performance = {}
    population=None
    generations={0:None}

In [14]:
pull_button = widgets.Button(description="Generation aktualisieren")
def pull_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        git_pull(name='all')
pull_button.on_click(pull_button_clicked)
    
generation_count = widgets.BoundedIntText(value=0, min=0, max=0)

def on_generation_count_change(change):
    log_widget.clear_output()
    with log_widget:
        df = model.generation_performance[change['new']]
        
    plot_accuracy_vs_name(df)
generation_count.observe(on_generation_count_change, names=['value'])    

generation_button = widgets.Button(description="Generation laden")

def generation_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        model.population = load_population()

        df = pandas.DataFrame([
            {'name': key,
             'loss': model.population[key].loss,
             'accuracy': model.population[key].accuracy
            } for key in model.population.keys()
        ])
        generation_index = max(model.generations.keys())+1
        model.generation_performance[generation_index] = df
        model.generations[generation_index] = model.population
        generation_count.max = generation_index
        generation_count.min = 1
        generation_count.value = generation_index
    
generation_button.on_click(generation_button_clicked)

evolve_button = widgets.Button(description="Evolution!")
def evolve_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        new_chromosomes = ga.evolve_population(list(model.population.values()))
        model.population = OrderedDict(zip(model.population.keys(), new_chromosomes))
evolve_button.on_click(evolve_button_clicked)

distribute_button = widgets.Button(description="Generation verteilen")
def distribute_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        for name,individuum in model.population.items():
            write_chromosome(name=name,chromosome=individuum)
            git_push(name)
distribute_button.on_click(distribute_button_clicked)

accuracy_plot_widget = widgets.Output(layout={'border': '1px solid black', 'width':'50%', 'height':'500px', 'scroll':'true'})
# generation_tree_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})
log_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})

def plot_accuracy_vs_name(df):
    accuracy_plot_widget.clear_output()
    with accuracy_plot_widget:
        fig, ax = pyplot.subplots(2,1,sharex=True,figsize=(7,8))
        df.iloc[::-1,::-1].plot.barh(x='name',y='accuracy', ax=ax[0],grid=True)
        df['accuracy'].plot.hist(ax=ax[1],grid=True)
        show_inline_matplotlib_plots()

def plot_generation_tree():
    pass

def update_generation_tree(df):
    pass

gui = widgets.VBox(children=[
    widgets.HBox([pull_button, generation_button]),
    generation_count,
    accuracy_plot_widget,
#     generation_tree_widget,
    evolve_button,
    distribute_button,
    log_widget,
])

In [15]:
display(gui)

In [16]:
for ind in model.population.values():
    print(ind.parent_a, ind.parent_b, ind.id)
    #ind.ml_model.summary()

AttributeError: 'NoneType' object has no attribute 'values'

In [ ]:
import newick

In [ ]:
# Generate tree
initial_population = model.generations[1]
nodes = newick.Node?

In [ ]:
for population in model.generations:
    p